In [3]:
import pandas as pd
import json
# from wip import WIP

class Job():
    def __init__(self, configure, processTime):
        self.configure = configure #fix
        self.processTime = processTime[processTime["RECIPE"] == self.configure["RECIPE"]] #fix
        self.startTime = 0 #var
        self.machineID = ''  #var
        self.endTime = 0 #var
         
        for i in self.configure.index:
            setattr(self, i, self.configure[i])

    def set_machine_id(self, machineID):
        self.machineID = machineID

    def set_start_time(self, time):
        self.startTime = time
        processTime = int(self.processTime[ self.processTime["EQP_ID"] == self.machineID ]["PROCESS_TIME"])
        self.endTime = self.startTime + processTime
    
    def get_end_time(self):
        return self.endTime
    
    def __str__(self): 
        return json.dumps(self.configure.to_dict(), indent=4)
    
    def __repr__(self): 
        return json.dumps(self.configure.to_dict(), indent=4)


In [11]:
class Machine(object):
    def __init__(self, configure, setupTime):
        self.configure = configure
        self.setupTime = setupTime

        self.jobs = []

    def add_job(self, job):
        self.jobs.append(job)

    def sort_jobs(self):
        currentTime = int(self.configure["RECOVER_TIME"])
        print(currentTime)

        for i in range(len(self.jobs)):
            self.jobs[i].set_start_time(currentTime)
            currentTime = self.jobs[i].get_end_time()
            
machines = []
for i in range(len(tool.values)):
    machines.append(Machine(tool.iloc[i], setup_time.values))  #setuptime整張表

In [6]:
wip = pd.read_excel("./semiconductor_data.xlsx", sheet_name=2, dtype=str) #K

In [4]:
eqp = pd.read_excel("./semiconductor_data.xlsx", sheet_name=0, dtype=str) #K

In [14]:
tool = pd.read_excel("./semiconductor_data.xlsx", sheet_name=1, dtype=str) #K

In [17]:
setup_time = pd.read_excel("./semiconductor_data.xlsx", sheet_name=3, usecols="B:CW") #欄位

In [15]:
jobs = []                               #K
for i in range(len(wip.values)):
    jobs.append(Job(wip.iloc[i], eqp)) 

In [18]:
machines = []
for i in range(len(tool.values)):
    machines.append(Machine(tool.iloc[i], setup_time.values))  #setuptime整張表

In [9]:
jobs[2].set_machine_id("EQP002") #工作派給機器

In [19]:
machines[1].add_job(jobs[2])

In [20]:
machines[1].sort_jobs()

240


TypeError: cannot convert the series to <class 'int'>

In [79]:
machines[1].jobs[0].get_end_time()

IndexError: list index out of range

In [80]:
RECIPE = pd.read_excel("./semiconductor_data.xlsx")

In [81]:
RECIPE

,EQP_ID,RECIPE,PROCESS_TIME
0,EQP001,X1000,80
1,EQP001,Y2000,140
2,EQP001,Z3000,80
3,EQP001,X4000,120
4,EQP001,Y5000,130
...,...,...,...
95,EQP010,Z6000,120
96,EQP010,X7000,150
97,EQP010,Y8000,110
98,EQP010,Z9000,130


In [73]:
for i in range(len(jobs)):                                                  #替代方法
    jobs[i].set_process_time(RECIPE[RECIPE["RECIPE"] == jobs[i].RECIPE])

In [74]:
jobs[0].processTime

,EQP_ID,RECIPE,PROCESS_TIME
7,EQP001,Y8000,100
17,EQP002,Y8000,120
27,EQP003,Y8000,130
37,EQP004,Y8000,110
47,EQP005,Y8000,130
57,EQP006,Y8000,150
67,EQP007,Y8000,110
77,EQP008,Y8000,150
87,EQP009,Y8000,140
97,EQP010,Y8000,110


In [75]:
jobs[0].RECIPE

'Y8000'

In [81]:
print(jobs[0])

{
    "LOT_ID": "LOT001",
    "OPE_NO": "2000.011",
    "EP": "1",
    "QTY": "8",
    "R_QT": "999",
    "ARRIV_T": "300",
    "STAGE": "STAGE",
    "URGENT_W": "0.5",
    "RECIPE": "Y8000",
    "CANRUN_TOOL": "EQP003EQP004EQP005EQP006EQP008EQP009",
    "FOUP": "FOUP0001"
}


In [19]:
setup_time = pd.read_excel("./semiconductor_data.xlsx", sheet_name=3, usecols="B:CW") #欄位

In [21]:
setup_time

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,0,10,10,10,10,10,10,10,10,10,...,10,10,10,10,0,10,0,10,10,10
1,10,0,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
2,10,10,0,10,0,10,10,10,10,10,...,10,10,10,10,10,10,10,0,10,10
3,10,10,10,0,10,0,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
4,10,10,0,10,0,10,10,10,10,10,...,10,10,10,10,10,10,10,0,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,10,10,10,10,10,10,10,10,10,0,...,10,10,10,10,10,0,10,10,10,10
96,0,10,10,10,10,10,10,10,10,10,...,10,10,10,10,0,10,0,10,10,10
97,10,10,0,10,0,10,10,10,10,10,...,10,10,10,10,10,10,10,0,10,10
98,10,10,10,10,10,10,10,10,10,10,...,0,10,10,10,10,10,10,10,0,0


In [37]:
setup_time = pd.read_excel("./semiconductor_data.xlsx", sheet_name=3) #欄位

In [38]:
setup_time

,Setup time,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,1,0,10,10,10,10,10,10,10,10,...,10,10,10,10,0,10,0,10,10,10
1,2,10,0,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
2,3,10,10,0,10,0,10,10,10,10,...,10,10,10,10,10,10,10,0,10,10
3,4,10,10,10,0,10,0,10,10,10,...,10,10,10,10,10,10,10,10,10,10
4,5,10,10,0,10,0,10,10,10,10,...,10,10,10,10,10,10,10,0,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,0,10,10,10,10
96,97,0,10,10,10,10,10,10,10,10,...,10,10,10,10,0,10,0,10,10,10
97,98,10,10,0,10,0,10,10,10,10,...,10,10,10,10,10,10,10,0,10,10
98,99,10,10,10,10,10,10,10,10,10,...,0,10,10,10,10,10,10,10,0,0


In [3]:
a=[1,2,3]
b=a
a[0]="s"
b[1]="k"
b

['s', 'k', 3]

In [9]:
x = 'Spam'
def func():
    x = 'HI!'
    return x
func()
print(func())

HI!
